Installing the Required Libraries

In [1]:
!pip install nibabel nilearn scikit-image scipy pyvista


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.7 MB/s eta 0:00:00


Code for the 3D Brain images in form of .ply

In [2]:
# ✅ Imports
import os, nibabel as nib, numpy as np
from nilearn.image import crop_img, resample_to_img
from scipy.ndimage import gaussian_filter, binary_fill_holes
from skimage import measure, morphology
from skimage.filters import threshold_li
from skimage.measure import label, regionprops
import pyvista as pv

# --- Load paths ---
flair_path = '/content/BraTS20_Training_001_flair.nii'
seg_path = '/content/BraTS20_Training_001_seg.nii'

# --- Load and crop images ---
flair_img = nib.load(flair_path)
seg_img = nib.load(seg_path)
cropped_flair_img = crop_img(flair_img)
flair_data = cropped_flair_img.get_fdata()
affine = cropped_flair_img.affine
cropped_seg_img = resample_to_img(seg_img, cropped_flair_img, interpolation='nearest')
seg_data = cropped_seg_img.get_fdata()

# --- Normalize and smooth FLAIR data ---
p2, p98 = np.percentile(flair_data, (2, 98))
flair_data = np.clip(flair_data, p2, p98)
flair_data = (flair_data - p2) / (p98 - p2)
smoothed = gaussian_filter(flair_data, sigma=1.0)

# --- Brain mask ---
threshold_val = threshold_li(smoothed)
initial_mask = smoothed > threshold_val
cleaned = morphology.binary_opening(initial_mask, morphology.ball(2))
cleaned = morphology.binary_closing(cleaned, morphology.ball(3))
cleaned = binary_fill_holes(cleaned)
labels = label(cleaned)
regions = regionprops(labels)
brain_mask = labels == max(regions, key=lambda x: x.area).label if regions else cleaned

# --- Preserve original brain_mask for surface ---
outer_brain_mask = brain_mask.copy()

# --- Fill only internal holes (not surface smoothing) ---
# Erode then dilate to preserve surface, only smooth interior
internal_fill = morphology.binary_erosion(brain_mask, morphology.ball(2))
internal_fill = binary_fill_holes(internal_fill)
internal_fill = morphology.binary_dilation(internal_fill, morphology.ball(2))

# Merge interior fill with original outer structure
brain_mask_filled = outer_brain_mask | internal_fill

# --- Tumor mask ---
tumor_mask = np.isin(seg_data.astype(int), [1, 2, 4]) & outer_brain_mask
tumor_mask = morphology.remove_small_objects(tumor_mask, min_size=30)

# --- Vessel mask ---
vessel_mask = (smoothed > 0.90) & outer_brain_mask
vessel_mask = morphology.remove_small_objects(vessel_mask.astype(bool), min_size=30)

# --- Mesh generator ---
def get_colored_mesh(mask, color_rgb, smooth_level=15):
    verts, faces, _, _ = measure.marching_cubes(mask.astype(np.float32), level=0.5)
    faces_vtk = np.hstack([np.full((faces.shape[0], 1), 3), faces]).astype(np.int64).flatten()
    mesh = pv.PolyData(verts, faces_vtk)
    mesh = mesh.clean(tolerance=1e-5)
    mesh = mesh.fill_holes(100.0)  # fill small leftover cracks
    mesh = mesh.smooth(n_iter=smooth_level, relaxation_factor=0.1)
    colors = np.tile(np.array(color_rgb, dtype=np.uint8), (mesh.n_points, 1))
    mesh.point_data['RGB'] = colors
    return mesh

# --- Create and combine meshes ---
brain_mesh = get_colored_mesh(brain_mask_filled.astype(bool), [150, 150, 255])  # Blue
vessel_mesh = get_colored_mesh(vessel_mask.astype(bool), [255, 80, 80])         # Red
tumor_mesh = get_colored_mesh(tumor_mask.astype(bool), [255, 0, 255])           # Pink

full_mesh = brain_mesh + vessel_mesh + tumor_mesh
tumor_only_mesh = tumor_mesh

# --- Save meshes ---
brain_mesh_path = '/content/brain_with_tumor_vessels.ply'
tumor_mesh_path = '/content/tumor_only.ply'

full_mesh.save(brain_mesh_path)
tumor_only_mesh.save(tumor_mesh_path)

print("✅ Exported with internal cracks filled and outer surface/tumor preserved:")
print(f"🧠 Full model → {brain_mesh_path}")
print(f"🎯 Tumor only → {tumor_mesh_path}")


✅ Exported with internal cracks filled and outer surface/tumor preserved:
🧠 Full model → /content/brain_with_tumor_vessels.ply
🎯 Tumor only → /content/tumor_only.ply


After getting the .ply file we have import it into Unity/Blender to view the 3D structure of the Brain and its Tumor.